# DrakeVisualizer.jl
This notebook demonstrates interacting with the Drake Visualizer app from Julia. On Mac and Ubuntu, just installing `DrakeVisualizer.jl` will also install a pre-built version of the standalone `drake-visualizer` executable. 

If you're not on one of those platforms, you'll need to install Director (which includes `drake-visualizer`) from <https://github.com/robotlocomotion/director>

In [ ]:
using Revise

In [ ]:
# Activate the DrakeVisualizer package, and import some other 
# useful functions
using DrakeVisualizer
using CoordinateTransformations
using Interact, Reactive
import GeometryTypes: HyperRectangle, Vec, HomogenousMesh
import ColorTypes: RGBA

In [ ]:
# First, we'll create a simple geometric object
box = HyperRectangle(Vec(0.,0,0), Vec(1.,1,1))

In [ ]:
# Visualizer() causes the viewer to spawn a geometry or a set of geometries. 
# It returns a Visualizer, which includes all the information
# about that loaded geometry. 
# Note that the model is initially loaded in the zero configuration 
# (that is, its position and rotation are all zeros). 
#
# New in version 0.3: Visualizer() now also automatically spawns a new
# viewer window. 

model = Visualizer(box)

In [ ]:
# We can use settransform!() to tell the viewer to draw the box at a specific
# position. Translation() creates a CoordinateTransformations.Transformation
# corresponding to the given x; y; z translation.
settransform!(model, Translation(1.,0,0))

In [ ]:
# We can also rotate the model by sending a different transformation
settransform!(model, LinearMap(AngleAxis(pi/4, 0, 0, 1)))

In [ ]:
# And we can clear the box
delete!(model)

In [ ]:
# and close the resulting window
close(model)

In [ ]:
# Now let's make some more complicated robots. We'll create a 
# new GeometryData from the box, but color it green this time.
green_box = GeometryData(box, RGBA(0., 1, 0, 0.5))
model = Visualizer(green_box)

In [ ]:
# and now let's delete it
delete!(model)

# The Viewer Tree

Now that we've introduced the basics of loading and transforming geometries, it's time to introduce the tree model used by DrakeVisualizer.jl. The idea is pretty simple:

* A Visualizer contains a *tree* of geometries
* We add, draw, and delete geometries by using their *path*, which is the path from the root of the tree to that geometry
* We can set the transform for any node in the tree. Setting the transform for a node in the tree affects all of its descendant geometries. 


In [ ]:
# First, we make an empty visualizer:
vis = Visualizer()

In [ ]:
# We can access a particular path within the visualizer with indexing notation:
vis[:group1]

In [ ]:
# We load geometries using the same path notation:
green_box_vis = setgeometry!(vis[:group1][:greenbox], green_box)

Check out the Scene Browser in the drake visualizer app. You should see folder:

    scene
      | remote tree viewer
          | group1
              | greenbox

  

In [ ]:
# And we set transforms in the same way:
settransform!(green_box_vis, Translation(0, 0, 1))

In [ ]:
# The same indexing notation makes it easy to get a handle to a 
# particular part of the viewer tree:
group1 = vis[:group1]
settransform!(group1, Translation(0, 0, -1))

The viewer tree becomes much more interesting (and useful) when we have multiple geometries. Let's add another geometry in the same group:

In [ ]:
setgeometry!(vis[:group1][:bluebox], GeometryData(box, RGBA(0, 0, 1, 0.5)))

The scene browser will now show:

    scene
      | remote tree viewer
          | group1
              | greenbox
              | bluebox

Since both the green and blue box are inside `group1`, we can move them together by transforming that whole group:

In [ ]:
settransform!(vis[:group1], Translation(1, 0, 0))

Or we can move just the green box:

In [ ]:
settransform!(vis[:group1][:greenbox], Translation(0, 1, 0))

The final pose of each geometry in the tree is just the composition of all of the transforms in the path from the root of the tree to that geometry. 

Let's try interactively moving the entire group (with the first slider) and also just the green box (with the second slider):

In [ ]:
@manipulate for x1 in linspace(0, 2), x2 in linspace(0, 2)
    batch(vis) do v
        settransform!(vis[:group1], Translation(x1, 0, 0))
        settransform!(vis[:group1][:greenbox], Translation(x2, 1, 0))
    end
end

Likewise, we can delete an entire group:

In [ ]:
delete!(vis[:group1])

In [ ]:
# Of course, we can draw much more interesting geometries than 
# just simple boxes. Let's load a 3D mesh and visualize it:
using MeshIO
using FileIO
cat_mesh = load(joinpath(Pkg.dir("GeometryTypes"), "test", "data", "cat.obj"))
setgeometry!(vis[:cat], cat_mesh)
settransform!(vis[:cat], LinearMap(AngleAxis(pi/2, 1, 0, 0)))

In [ ]:
delete!(model)

In [ ]:
# Next, let's create a triangulated mesh by finding
# the 0-level set of some function. 
# 
# First, we'll define our function:
f = x -> sum(sin, 5 * x)

# Then we pick a region of interest in which to sample the function.
# This region starts at (-1, -1, -1) and extends to (1, 1, 1):
lower_bound = Vec(-1.,-1,-1)
upper_bound = Vec(1., 1, 1)

# Those two pieces of information are all we need to construct a robot
# geometry. For this, we'll need the contour_mesh function:
mesh = contour_mesh(f, lower_bound, upper_bound)
# Under the hood, this will sample f at regularly spaced points inside
# the bounding rectangle, then compute a surface that connects all the 
# points for which f(x) = 0.

# And now we can load that geometry into the visualizer
model = Visualizer(mesh)

In [ ]:
# We can even manipulate the geometry by changing the iso level. 
# By default, contour_mesh constructs a mesh connecting the points 
# in space for which f(x) = 0, where 0 is called the isosurface level
# or iso level. But we can change that iso level to any number we want:

f = x -> sum(sin, 5 * x)
lower_bound = Vec(-1.,-1,-1)
upper_bound = Vec(1., 1, 1)

@manipulate for iso_level in linspace(-1, 1, 51)
    geometry = contour_mesh(f, lower_bound, upper_bound, iso_level)
    setgeometry!(model, geometry)
end

# Note that for high iso_level values our geometry gets cut off at the
# edges. We could fix that by replacing the bounds with a bigger box. 

In [ ]:
delete!(model)

# Batching for Better Performance

Each time you call `setgeometry!()`, `settransform!()`, or `delete!()`, there is some communication between Julia and the Drake Visualizer application. That communication can take time, so if you are loading or drawing many geometries simultaneously, then you may want to batch those operations. Batching is easy. Instead of doing this:


In [ ]:
vis = Visualizer()
setgeometry!(vis[:group1][:box1], green_box)
setgeometry!(vis[:group1][:box2], green_box)
settransform!(vis[:group1], Translation(0, 1, 0))
settransform!(vis[:group1][:box1], Translation(1, 0, 0))

Just put all the commands inside a call to `batch()`:

In [ ]:
vis = Visualizer()
batch(vis) do v
    setgeometry!(v[:group1][:box1], green_box)
    setgeometry!(v[:group1][:box2], green_box)
    settransform!(v[:group1], Translation(0, 1, 0))
    settransform!(v[:group1][:box1], Translation(1, 0, 0))
end

The above syntax will perform exactly the same drawing operations, but will only send one message to the viewer at the end of the `batch()` call. 

In [ ]:
delete!(vis)

# More Geometry Types

`DrakeVisualizer` supports a few extra types of geometry that you may want to visualize. One such geometry is a point cloud: 

In [ ]:
# A PointCloud can be constructed from a vector of points.
# The easiest way to represent a point is just a standard Julia vector:
pointcloud = PointCloud([[x, 0, 0] for x in linspace(-1, 1)])
setgeometry!(vis[:pointcloud], pointcloud)

# For more efficient point clouds, try using a vector of
# StaticArrays.SVector types. 

In [ ]:
using ColorTypes: RGB

In [ ]:
# PointClouds can also have "channels" which describe their
# data. One useful channel is the :rgb channel:
delete!(vis[:pointcloud])
pointcloud.channels[:rgb] = [RGB(0, g, 0) for g in linspace(0, 1)]
setgeometry!(vis[:pointcloud], pointcloud)

In [ ]:
delete!(vis)

In [ ]:
# Close the viewer:
close(vis)